In [6]:
from visiprog.data import *
from visiprog.metriclearning import *
from visiprog.evaluate import *
%matplotlib inline
import numpy as np
from visiprog.metric_learn.fda_visiprog import LFDA_VISIPROG
from metric_learn import Covariance

from IPython.display import Image, display
from sklearn.model_selection import train_test_split
import pandas as pd

from sklearn.cluster import KMeans

In [7]:
raw_feature = np.genfromtxt('visiprog/data/curetaliasfree.csv',delimiter=",")
label_material = read_material_label()
groups_VSP, material = read_VSP_group(pappas_only=False)

D = count_matrix(groups_VSP, raw_feature.shape[0])
clusters_VSP = spectralClusteringDBSCAN(D, threshold=0,eps=0.001)

print(raw_feature.shape)
print(label_material.shape)

(5245, 82)
(5245,)


In [8]:
label_visiprog = get_VSP_label(groups_VSP, raw_feature.shape[0])
label_VSP_clusters = get_VSP_label(clusters_VSP, raw_feature.shape[0])

In [48]:
label_visiprog = get_VSP_label(groups_VSP, raw_feature.shape[0])


In [9]:
model = Covariance()
model.fit(raw_feature)
X_STSIMM = model.transform(raw_feature)
# print(X_STSIMM)

In [52]:
accuracy_STSIMM = []
accuracy_mat = []
accuracy_vsp_original = []
accuracy_vsp = []

n_cluster = 5

N_material = 61
N_material_train = 20

for fold in range(10):
    print("Trial {}".format(fold))

    # randomly select N_material_train material for training the metric
    # and test on the rest
    train_select = np.zeros_like(label_material).astype(bool)

    idx_train = np.array([])
    for i in range(N_material_train):
        train_class = np.random.randint(0, N_material)
        train_select |= (label_material == train_class)

    idx_train = np.where(train_select == True)[0]
    idx_test = np.where(train_select == False)[0]

    idx_train = idx_train.astype(int)
    idx_test = idx_test.astype(int)

    print("Training shape {} and test shape {}".format(len(idx_train), len(idx_test)))

    X_train = raw_feature[idx_train,:]
    X_STSIMM_train = X_STSIMM[idx_train,:]
    Y_VSP_train = label_visiprog[idx_train]
    Y_mat_train = label_material[idx_train]

    X_test = raw_feature[idx_test,:]
    X_STSIMM_test = X_STSIMM[idx_test,:]
    Y_VSP_test = label_visiprog[idx_test]
    Y_mat_test = label_material[idx_test]

    res_STSIMM = kmean_subclass(X_STSIMM_test, Y_mat_test, X_STSIMM_test, Y_mat_test, n_cluster)
    accuracy_STSIMM.append(res_STSIMM['accuracy'])

    model = LFDA()
    try:
        model.fit(X_train, Y_mat_train)
        X_mat_test = model.transform(X_test)
        res = kmean_subclass(X_mat_test, Y_mat_test, X_mat_test, Y_mat_test, n_cluster)
        accuracy_mat.append(res['accuracy'])
    except Exception as e:
        print(e)
        acuracy_mat.append(np.nan)

    model_vsp_original = LFDA()
    try:
        # remove the unlabeled data
        idx_outlier = np.where(Y_VSP_train == 0)[0]
        X_train_without_outlier = X_train[~idx_outlier,:]
        Y_VSP_train_without_outlier = Y_VSP_train[~idx_outlier]
        
        model_vsp_original.fit(X_train_without_outlier, Y_VSP_train_without_outlier)
        X_vsp_original_train = model_vsp_original.transform(X_train)
        X_vsp_original_test = model_vsp_original.transform(X_test)
        res_vsp_original = kmean_subclass(X_vsp_original_test, Y_mat_test, \
                                          X_vsp_original_test, Y_mat_test, n_cluster)
        accuracy_vsp_original.append(res_vsp_original['accuracy'])
    except Exception as e:
        print(e)
        accuracy_vsp_original.append(np.nan)

    model_vsp = LFDA_VISIPROG()
    try:
        model_vsp.fit(X_train, Y_VSP_train)
        X_vsp_test = model_vsp.transform(X_test)
        res_vsp = kmean_subclass(X_vsp_test, Y_mat_test, X_vsp_test, Y_mat_test, n_cluster)
        accuracy_vsp.append(res_vsp['accuracy'])
    except Exception as e:
        print(e)
        accuracy_vsp.append(np.nan)


df = pd.DataFrame({'visiprog':accuracy_vsp, 'material':accuracy_mat, 'STSIMM':accuracy_STSIMM, \
       'visiprog_original':accuracy_vsp_original}, columns=['STSIMM','material','visiprog_original','visiprog'])

df.loc['average'] = df.mean()

display(df)
# display(df.mean())

df.to_csv('thrasos/across_trainedongroups_20material_k{}.csv'.format(n_cluster))

Trial 0
Training shape 1518 and test shape 3727
Trial 1
Training shape 1190 and test shape 4055
Trial 2
Training shape 1467 and test shape 3778
Trial 3
Training shape 1432 and test shape 3813
Trial 4
Training shape 1596 and test shape 3649
Trial 5
Training shape 1370 and test shape 3875
Trial 6
Training shape 1627 and test shape 3618
Trial 7
Training shape 1527 and test shape 3718
Trial 8
Training shape 1542 and test shape 3703
Trial 9
Training shape 1444 and test shape 3801


,STSIMM,material,visiprog_original,visiprog
0,0.963241,0.973974,0.944996,0.965924
1,0.959063,0.952404,0.893958,0.935142
2,0.956326,0.953150,0.946003,0.948385
3,0.958038,0.960661,0.897194,0.934435
4,0.954508,0.974788,0.932036,0.961085
5,0.963871,0.958452,0.918710,0.925935
6,0.948590,0.948038,0.925650,0.957435
7,0.959656,0.954276,0.912856,0.941366
8,0.963003,0.960032,0.945990,0.937888
9,0.962115,0.945541,0.898974,0.935806
